## Proje Dokümantasyonu

### 1. Giriş ve Veri Seti Açıklaması
- Bu bölümde, projenin amacı, kapsamı ve kullanılan veri setleri hakkında genel bilgiler sunulur.

### 2. Veri Hazırlığı
- Veri setinin okunması, temizlenmesi ve analize uygun hale getirilmesi süreçleri detaylandırılır.

### 3. Çözümle İlgili Genel Yaklaşım
- Projede izlenen strateji ve çözümün genel hatları açıklanır.

### 4. Churn Merchants
#### Churn Merchants Tespiti ve Tahminlenmesi:
- Churn olan müşterilerin tespit edilmesi ve bu müşterilere ilişkin tahminlerin yapılma yöntemleri anlatılır.

### 5. Full Merchants
#### Full Merchants Tespiti ve Modellenmesi:
- Projede "Full Merchant" olarak tanımlanan müşteri grubunun nasıl tespit edildiği ve bu grup üzerinde yapılan modelleme işlemleri detaylandırılır.

### 6. Active Merchants
#### Active Merchants Tespiti ve Kural Tabanlı Sistemin Uygulanması:
- Aktif müşterilerin nasıl belirlendiği ve bu müşterilere özel uygulanan kural tabanlı sistem anlatılır.

### 7. Sonuçların Birleştirilmesi ve Kaydedilmesi
- Farklı müşteri grupları için elde edilen sonuçların nasıl birleştirildiği ve sonuçların nasıl kaydedildiği üzerine bilgiler verilir.


#

## 1. Giriş ve Veri Seti Açıklaması

Bu notebook, iyzico ve Coderspace işbirliğiyle düzenlenen iyzico Datathon yarışmasındaki **Public LB'da 3. ve Private LB'da 4.** olarak yarışmayı tamamladığım çalışmadır. iyzico, 2013 yılında kurularak finansal hizmetleri daha erişilebilir hale getirmeyi amaçlamıştır. Yapay zeka tabanlı sanal POS ve ödeme teknolojileri ile 100 binden fazla üye iş yeriyle çalışmış ve alternatif ödeme yöntemleri sunarak alışverişi kolaylaştırmıştır.

Bu çalışmada, iyzico'nun sağladığı veri setini kullanarak 2023 yılının son çeyreği için iş yerlerinin işlem sayılarını (net_payment_count) tahmin edeceğim. Eğitim veri seti (train.csv), 2020'nin başından 2023'ün Eylül ayına kadar aylık işlem sayılarını içermektedir.

Veri seti şu bilgileri içerir:
- **merchant_id**: Maskelenmiş iş yeri ID'si.
- **month_id**: İşlemin yapıldığı ay, YYYYMM formatında.
- **net_payment_count**: İş yerinin ay içindeki net işlem sayısı.

Verisetinde toplamda 8 kolon bulunmaktadır. Veri analizi sürecinde bu bilgileri kullanarak çalışmayla alakalı bir strateji geliştirdim. Veri analizim sonrası modelleme ve tahmin sürecinde sadece 3 kolonu kullandım. Kurallar gereği veri seti, çalışmaların çıktıları ve veri analizi sürecini public olarak paylaşamamaktayım. Yukarıdaki data card, yarışma kapsamında geliştirdiğim modelin anlaşılabilirliğini artırmak için veri setindeki kolonların açıklamalarını içermekte olup, yalnızca eğitimsel ve bilgi paylaşım amaçlıdır ve veri setindeki kolonların sadece bir kısmıdır.

Bu çalışma, yarışma içerisinde; **Public Score'u 28.87614 (LB: 3/328 Teams)** , **Private Score'u 30.74141 (LB: 4/328 Teams)** olan tahminin elde edildiği çalışmadır.


##

## 2. Veri Hazırlığı

In [ ]:
# Gerekli kütüphanelerin import edilmesi

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
import itertools
import warnings
from tqdm import tqdm
from numpy.linalg import LinAlgError

In [ ]:
# Veri setinin yüklenmesi

def load_data(file_name):
    data = pd.read_csv(file_name)
    return data

In [ ]:
# Veri setinin pivot tablo olarak hazırlanması

def prepare_pivot_table(data):
    data['month_id'] = data['month_id'].astype(str)
    pivot_table = data.pivot_table(index='merchant_id', 
                                   columns='month_id', 
                                   values='net_payment_count',
                                   aggfunc='sum').reset_index()
    return pivot_table



In [ ]:
# Pivot tablosunun sütun başlıklarını düzeltilmesi

def flatten_pivot_table_columns(pivot_table):
    pivot_table = pivot_table.reset_index()

    new_columns = []
    for col in pivot_table.columns:
        if isinstance(col, tuple):
            new_columns.append('_'.join(str(lev) for lev in col if str(lev) != ''))
        else:
            new_columns.append(col)
    pivot_table.columns = new_columns

    return pivot_table

In [ ]:
data_file_name = 'train.csv'
data = load_data(data_file_name)
pivot_table = prepare_pivot_table(data)
pivot_table = flatten_pivot_table_columns(pivot_table)

In [ ]:
pivot_table

##

## 3. Çözümle İlgili Genel Yaklaşım

Yaptığım veri analizi sonucunda, verisetimi **3 parça** halinde incelemeye karar verdim.

1. **parça churn müşterilerimiz,**
2. **parça kesintisiz alım yapan ve en yenisi son 24 aydır müşterimiz olan full müşterilerimiz,**
3. **parça churn olmayan fakat kesintisiz alım yapmayan active müşterilerimiz.**

Bu 3 ayrı parçaya birbirinden farklı yaklaşımlarla tahmin süreçleri kurdum.

#

## 4. Churn Merchants

### Churn Olan Merchant_id'lerin Tespiti:

- Bu bölümde, **son üç ay** içerisinde herhangi bir aktivitesi olmayan ve bu durumuyla potansiyel olarak churn olmuş merchant'ların tespiti yapılmaktadır. 
- Bu işlem için son üç ayı temsil eden sütun isimleri (**202307, 202308, 202309**) kullanılmakta ve bu dönemlerde NaN değerlere sahip merchant_idler belirlenmektedir. Bu merchant_idler bir liste halinde (churn_list) saklanmaktadır. 
- Tespit edilen bu merchant_idlere ait kayıtlar daha sonra ele alınarak, gelecek üç ay (**202310, 202311, 202312**) için tahminler **0** olarak belirlenmektedir. 

In [ ]:
def detect_churn_merchants(pivot_table):
    
    # Geçmiş 3 ay için churn merchant'ların tespit edilmesi
    last_three_months_columns = ['202307', '202308', '202309']
    churn_list = pivot_table[pivot_table[last_three_months_columns].isna().all(axis=1)]['merchant_id']
    churn_merchants = pivot_table[pivot_table['merchant_id'].isin(churn_list)]
    
    # Gelecek 3 ay için tahmin sütunlarını oluşturma ve 0 olarak atama
    prediction_months = ["202310", "202311", "202312"]
    for month in prediction_months:
        churn_merchants.loc[:, month] = 0
    
    return churn_merchants


churn_merchants = detect_churn_merchants(pivot_table)

#

## 5. Full Merchants

### Model Genel Bakış

Modelin uygulama süreci aşağıdaki adımlar izlenerek gerçekleştirilmiştir:

1. **Veri Hazırlığı:** Churn olmuş merchant'lar veri setinden çıkarılmıştır. Daha sonra, kalan veri üzerinde filtreleme işlemi yapılmıştır. Filtreleme, son 45 aydır müşteri olan ve hiç NaN değeri bulunmayan merchant'lar ile başlayıp, süreç son 24 aydır müşteri olan ve hiç NaN değeri bulunmayan merchant'lar ile bitirilmiştir. Bu süreçte model uygulanan tüm merchant'lar, 'full merchant' olarak değerlendirilmiştir.

2. **Model İçeriği:** Merchant'lara ARIMA ve SARIMAX modelleri uygulanmıştır. Bu modeller için `p, d, q` ve `P, D, Q` parametreleri için belirlenen aralıklar `range(0,2)`, mevsimsel etkiler için ise `range(11,13)` olarak ayarlanmıştır. Bu belirlenen aralıklar üzerinden, grid-search yöntemi ile hiperparametre optimizasyonu yapılmıştır. 

3. **Hiperparametre Optimizasyonu ve Model Seçimi:** Parametreler ve tahminler için DataFrame'ler oluşturulmuştur. ARIMA ve SARIMAX modelleri için hiperparametre optimizasyonu gerçekleştiren bir fonksiyon yazılmıştır. Bu fonksiyon, her merchant için en düşük ortalama MAE'yi sağlayan hiperparametreleri seçer. Modellerin performansı karşılaştırılarak, en düşük MAE'yi veren model ve bu modeldeki hiperparametreler belirlenir. Bu model, final model olarak kabul edilir ve merchant'ın tahminleri bu model üzerinden alınır.

4. **Cross-Validation (Çapraz Doğrulama):** Toplamda 6 fold aşağıdaki gibi tanımlanmıştır:

   - `(data[:-3], data[-3:])`,   # 1. fold (202307-202309)
   - `(data[:-4], data[-4:-1])`, # 2. fold (202306-202308)
   - `(data[:-5], data[-5:-2])`, # 3. fold (202305-202307)
   - `(data[:-6], data[-6:-3])`, # 4. fold (202304-202306)
   - `(data[:-7], data[-7:-4])`, # 5. fold (202303-202305)
   - `(data[:-8], data[-8:-5])`, # 6. fold (202302-202304)

Model, üç aşamalı bir süreçten geçirilmiştir:

1. İlk aşamada, ilk fold için hiperparametre optimizasyonu yapılarak final model ile tahmin alınır.
2. İkinci aşamada, ilk 3 fold için ortalama MAE'ye göre hiperparametre optimizasyonu yapılarak final model ile tahmin alınır.
3. Üçüncü aşamada, tüm foldlar için, yani 6 fold için ortalama MAE'ye göre hiperparametre optimizasyonu yapılarak final model ile tahmin alınır.

Bir merchant, süreç boyunca ilk olarak ARIMA, sonra SARIMAX ve son olarak yine SARIMAX modelini final model olarak seçebilir. Son tahminler, bu üç süreçten elde edilen tahminlerin ortalaması alınarak ve 202310, 202311 ve 202312 ayları için final tahminler elde edilir.

In [ ]:
# Verisetin'den tahmini tamamlanan churn merchantların çıkarılması

active_merchants = pivot_table[~pivot_table['merchant_id'].isin(churn_list)] 
active_merchants = active_merchants.set_index('merchant_id')

### Modelin Uygulanması

In [ ]:
full_merchants = active_merchants.copy()

# SARIMA ve ARIMA hiperparametreleri tanımlanır.
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

# Parametrelerin ve tahminlerin saklanması için DataFrame'ler oluşturulur.
params = pd.DataFrame(columns=['merchant_id', 'process', 'model_type', 'best_order', 'best_seasonal_order', 'best_mae'])
forecast_months = ["202310", "202311", "202312"]

# Model optimizasyon fonksiyonu tanımlanır.
def model_optimizer_mae(train_test_sets, orders, seasonal_orders=None, is_sarimax=False):
    best_mae, best_params, best_seasonal_params = float("inf"), None, None
    for order in orders:
        if is_sarimax:
            for seasonal_order in seasonal_orders:
                total_mae = 0
                try:
                    for train, test in train_test_sets:
                        model = SARIMAX(train, order=order, seasonal_order=seasonal_order)
                        model_fit = model.fit(disp=0)
                        y_pred = model_fit.get_forecast(steps=len(test)).predicted_mean
                        total_mae += mean_absolute_error(test, y_pred)
                    avg_mae = total_mae / len(train_test_sets)
                    if avg_mae < best_mae:
                        best_mae, best_params, best_seasonal_params = avg_mae, order, seasonal_order
                except (LinAlgError, ValueError):
                    continue
        else:  # ARIMA
            try:
                total_mae = 0
                for train, test in train_test_sets:
                    model = ARIMA(train, order=order)
                    model_fit = model.fit()
                    y_pred = model_fit.forecast(steps=len(test))
                    total_mae += mean_absolute_error(test, y_pred)
                avg_mae = total_mae / len(train_test_sets)
                if avg_mae < best_mae:
                    best_mae, best_params = avg_mae, order
            except (LinAlgError, ValueError):
                continue
                
    return best_mae, best_params, best_seasonal_params

# Süreçlerin tanımlanması ve iterasyonları gerçekleştirilir.
process_names = {1: "1_fold", 3: "3_fold", 6: "6_fold"}

# Final tahminlerin saklanması için bir sözlük oluşturulur.
final_predictions = {}

# Filtreleme ve tahmin süreçleri
for months_back in range(45, 23, -1):
    
    active_filter = active_merchants.iloc[:, -months_back:].dropna(thresh=months_back)
    filtered_merchants = active_filter.index.tolist()

    for merchant_id in tqdm(filtered_merchants):
        data = active_filter.loc[merchant_id].values
        temp_predictions = {month: [] for month in forecast_months}

        for process, process_name in process_names.items():
            
            # Fold'lar tanımlanır.
            train_test_sets = [(data[:-n], data[-n:]) for n in range(3, 3 + process)]
            
            # SARIMAX ve ARIMA için optimizasyonlar gerçekleştirilir.
            sarimax_mae, sarimax_best_order, sarimax_best_seasonal_order = model_optimizer_mae(
                train_test_sets, pdq, seasonal_pdq, is_sarimax=True
            )
            arima_mae, arima_best_order, _ = model_optimizer_mae(
                train_test_sets, pdq, is_sarimax=False
            )

            
            # En iyi modelle tahmin yapılır.
            if sarimax_mae < arima_mae:
                final_model = SARIMAX(data, order=sarimax_best_order, seasonal_order=sarimax_best_seasonal_order)
                final_model_fit = final_model.fit(disp=0)
                forecast = final_model_fit.get_forecast(steps=3).predicted_mean
                model_type = 'SARIMAX'
                best_order, best_seasonal_order, overall_best_mae = sarimax_best_order, sarimax_best_seasonal_order, sarimax_mae
            else:
                final_model = ARIMA(data, order=arima_best_order)
                final_model_fit = final_model.fit()
                forecast = final_model_fit.forecast(steps=3)
                model_type = 'ARIMA'
                best_order, best_seasonal_order, overall_best_mae = arima_best_order, None, arima_mae
            
            # Her bir süreç için parametreler ve tahminler saklanır.
            params_row = {
                'merchant_id': merchant_id,
                'process': process_name,
                'model_type': model_type,
                'best_order': str(best_order),
                'best_seasonal_order': str(best_seasonal_order) if best_seasonal_order is not None else None,
                'best_mae': overall_best_mae
            }
            params = pd.concat([params, pd.DataFrame([params_row])], ignore_index=True)
            
            for i, month in enumerate(forecast_months):
                temp_predictions[month].append(forecast[i])

        # Her bir ay için tahminlerin ortalamasını al ve final_predictions sözlüğünde sakla
        final_predictions[merchant_id] = {month: np.mean(temp_predictions[month]) for month in forecast_months}

        # final_predictions sözlüğündeki tahminleri kullanarak full_merchants DataFrame'ini güncelle
        for merchant_id, predictions in final_predictions.items():
            for month, prediction in predictions.items():
                full_merchants.loc[merchant_id, month] = prediction
    
    # Filtrelenen merchant_id'leri çıkar
    active_merchants = active_merchants[~(active_merchants.index.isin(filtered_merchants))]

#

## 6. Active Merchants

### Kural Tabanlı Sistemin Uygulanması: Full Merchants

Tespit edilen aktif merchant'lara uygulanacak kural tabanlı sistemin aşamaları şu şekildedir:

#### Kural Tabanlı Sisteme Genel Bakış

1. **Veri Seti Hazırlığı:**
   - `active_merchants`: İlk etapta Churn olmayan merchant_id'lerini içeren bir DataFrame.
   - `full_merchants`: Model uygulanan merchant_id'lerini içeren bir DataFrame.
   - Güncellenmiş `active_merchants`: Model uygulanan merchant_id'ler DataFrame'den çıkartılarak uygulama yapıldığı için DataFrame'de kalan kişiler bizim için kural tabanlı sistemin uygulanacağı, güncellenmiş merchant_id'lerini barındıran bir DataFrame.

#### Kural Tabanlı Sistemin Uygulanması

1. **İşlem Hacmindeki Yüzdesel Değişim:**
   - 202307'den 202308'e ve 202308'den 202309'a geçişteki işlem hacmindeki yüzdesel değişim hesaplanır.

2. **Ortalama Değişimin Hesaplanması ve Limitler:**
   - Hesaplanan yüzdesel değişimlerin ortalaması alınır. Ancak, dengesizliği önlemek için -%8 ve %8 olarak belirlenen alt ve üst limitler uygulanır.

3. **Ortalama Değişime Göre Tahminler:**
   - Merchant'ın ortalama değişimi belirlenen aralıklar içindeyse, 202309 ayındaki işlem hacmi bu ortalama değişim oranıyla güncellenir. Bu güncellenmiş işlem hacmi, 202310, 202311 ve 202312 aylarının tahminleri için kullanılır.

4. **Limit Aşımı Durumunda Uygulanacak İşlemler:**
    - Eğer alt limit aşılmışsa, yani ortalama değişim -%8'den küçükse, bu durumda 202309 ayındaki işlem hacmine alt limit kadar değişim uygulanır. Yani 202309 ayındaki işlem hacmi 0.92 ile çarpılır ve elde edilen işlem hacmi, 202310, 202311 ve 202312 aylarının tahminlerini oluşturmak için kullanılır.
   - Eğer üst limit aşılmışsa, yani ortalama değişim %8'den büyükse, bu durumda 202309 ayındaki işlem hacmine üst limit kadar değişim uygulanır. Yani 202309 ayındaki işlem hacmi 1.08 ile çarpılır ve elde edilen işlem hacmi, 202310, 202311 ve 202312 aylarının tahminlerini oluşturmak için kullanılır.

Bu yöntemle, işlem hacmindeki değişimlerin ortalaması alınarak, alt ve üst sınırlar belirlenir ve bu sınırlar içinde kalan merchant’lara ortalama değişim uygulanarak tahminler oluşturulur. Ancak, alt veya üst sınırları aşan merchant’lar için bu sınırlara uygun şekilde işlem hacmi düzeltilir ve tahminler bu düzeltilmiş veriler üzerinden yapılır.

In [ ]:
def calculate_forecasts(active_merchants):
   
    # İşlem hacmindeki yüzdesel değişim hesaplanır
    percent_change_1 = (active_merchants['202308'] - active_merchants['202307']) / active_merchants['202307'] * 100
    percent_change_2 = (active_merchants['202309'] - active_merchants['202308']) / active_merchants['202308'] * 100

    # Ortalama yüzdesel değişim hesaplanır
    average_percent_change = (percent_change_1.mean() + percent_change_2.mean()) / 2

    # Alt ve üst limitler belirlenir
    lower_limit = -8
    upper_limit = 8

    # İşlem hacmi tahminlerinin hesaplanması
    if lower_limit <= average_percent_change <= upper_limit:
        forecast_multiplier = 1 + average_percent_change / 100
    else:
        if average_percent_change < lower_limit:
            forecast_multiplier = 1 + lower_limit / 100
        else:
            forecast_multiplier = 1 + upper_limit / 100

    # Tahminler için kullanılmak üzere işlem hacmi 202310, 202311 ve 202312 aylarına taşınır
    active_merchants['202310'] = active_merchants['202309'] * forecast_multiplier
    active_merchants['202311'] = active_merchants['202309'] * forecast_multiplier
    active_merchants['202312'] = active_merchants['202309'] * forecast_multiplier

    return active_merchants

active_merchants = calculate_forecasts(active_merchants)

#

## 7. Sonuçların Birleştirilmesi ve Kaydedilmesi

- **CHURN MERCHANTS**'ların prediction'ları **0,0,0** atanmaktadır.
- **FULL MERCHANTS**'ların prediction'ları **modelden** elde edilmektedir. 
- **ACTIVE MERCHANTS**'ların prediction'ları **kural tabanlı sistem** ile elde edilmektedir.

In [ ]:
# CHURN, FULL ve ACTIVE MERCHANTS'ların prediction'larını birleştirme
merged_predictions = pd.concat([churn_merchants, full_merchants, active_merchants], axis=0)
merged_predictions.reset_index(drop=True, inplace=True)          
 
merged_predictions.to_csv("merged_predictions.csv", index=False)